In [1]:
import keras
import keras.backend as K
from keras.datasets import mnist
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Dropout, Activation
from keras.optimizers import RMSprop
from keras.callbacks import CSVLogger
import os
import keras.activations
import time

C:\Users\MaksutovRN\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

num_classes = 10

x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

60000 train samples
10000 test samples


In [3]:
def standardize(train, test):
    mean, std = train.mean(), train.std()
    train = (train - mean) / std
    test = (test - mean) / std
    return train, test

In [5]:
x_train, x_test = standardize(x_train, x_test)



In [6]:
batch_size = 64
epochs = 100
units = 64
experiments = 5
start = 0
# activations = ['sigmoid', 'tanh', 'relu', 'linear', 'elu', 'softplus', 'softsign', 'hard_sigmoid', 'LeakyReLU', 'ThresholdedReLU']
activations = ['ThresholdedReLU']

In [9]:
start_time = time.time()
counter = 0
total_items = len(activations) * experiments

for act in activations:
    print("Training for activation " + act + " with RMSProp optimizer")

    for i in range(experiments):

        K.clear_session()
        K.reset_uids()

        act_dict = {
          'sigmoid': Activation(keras.activations.sigmoid),
          'tanh': Activation(keras.activations.tanh),
          'relu': Activation(keras.activations.relu),
          'linear': Activation(keras.activations.linear),
          'elu': Activation(keras.activations.elu),
          'softplus': Activation(keras.activations.softplus),
          'softsign': Activation(keras.activations.softsign),
          'hard_sigmoid': Activation(keras.activations.hard_sigmoid),
          'LeakyReLU': keras.layers.advanced_activations.LeakyReLU(),
          'selu': Activation(keras.activations.selu),
          'ThresholdedReLU': keras.layers.advanced_activations.ThresholdedReLU(theta=0.0) 
        }

        model_name = 'std_' + act + '0.0_rmsp_' + str(i + start) + '_' + str(units)
        inputs = Input(shape=(784,))
        x = Dense(units)(inputs)
        x = act_dict[act](x)
        x = Dropout(0.2)(x)
        x = Dense(units)(x)
        x = act_dict[act](x)
        x = Dropout(0.2)(x)
        predictions = Dense(num_classes, activation='softmax')(x)
        model = Model(inputs=inputs, outputs=predictions)

        model.compile(loss='categorical_crossentropy',
                          optimizer=RMSprop(lr=0.001),
                          metrics=['accuracy'])

        csv_logger = CSVLogger('./training_logs/%s.csv' % (model_name), append=False)
        history = model.fit(x_train, y_train,
                            batch_size=batch_size,
                            epochs=epochs,
                            verbose=1,
                            validation_data=(x_test, y_test), callbacks=[csv_logger])

        score = model.evaluate(x_test, y_test, verbose=0)
        print('Test loss:', score[0])
        print('Test accuracy:', score[1])

        t = time.time()
        time_diff = t - start_time
        counter +=1
        rem_items = total_items - counter
        total_time = round((total_items / counter) * time_diff)
        rem_time = round(total_time - time_diff)
        m, s = divmod(rem_time, 60)
        h, m = divmod(m, 60)
        d, h = divmod(h, 24)
        print('Remaining time: %d days %d hours %02d minutes %02d seconds' % (d, h, m, s))

Training for activation ThresholdedReLU with RMSProp optimizer
Train on 60000 samples, validate on 10000 samples
Epoch 1/100
60000/60000 [==============================] - 4s 64us/step - loss: 0.4338 - acc: 0.8676 - val_loss: 0.1688 - val_acc: 0.9531
Epoch 2/100
60000/60000 [==============================] - 3s 57us/step - loss: 0.2309 - acc: 0.9334 - val_loss: 0.1328 - val_acc: 0.9616
Epoch 3/100
60000/60000 [==============================] - 3s 57us/step - loss: 0.1893 - acc: 0.9463 - val_loss: 0.1219 - val_acc: 0.9668
Epoch 4/100
60000/60000 [==============================] - 3s 57us/step - loss: 0.1724 - acc: 0.9504 - val_loss: 0.1146 - val_acc: 0.9674
Epoch 5/100
60000/60000 [==============================] - 4s 60us/step - loss: 0.1659 - acc: 0.9537 - val_loss: 0.1145 - val_acc: 0.9691
Epoch 6/100
60000/60000 [==============================] - 3s 56us/step - loss: 0.1574 - acc: 0.9567 - val_loss: 0.1133 - val_acc: 0.9695
Epoch 7/100
60000/60000 [==============================] - 

60000/60000 [==============================] - 3s 53us/step - loss: 0.1439 - acc: 0.9715 - val_loss: 0.1959 - val_acc: 0.9709
Epoch 60/100
60000/60000 [==============================] - 3s 50us/step - loss: 0.1493 - acc: 0.9711 - val_loss: 0.1899 - val_acc: 0.9709
Epoch 61/100
60000/60000 [==============================] - 3s 51us/step - loss: 0.1458 - acc: 0.9703 - val_loss: 0.1906 - val_acc: 0.9711
Epoch 62/100
60000/60000 [==============================] - 3s 51us/step - loss: 0.1491 - acc: 0.9694 - val_loss: 0.1905 - val_acc: 0.9723
Epoch 63/100
60000/60000 [==============================] - 3s 53us/step - loss: 0.1485 - acc: 0.9706 - val_loss: 0.2036 - val_acc: 0.9698
Epoch 64/100
60000/60000 [==============================] - 3s 51us/step - loss: 0.1561 - acc: 0.9689 - val_loss: 0.2012 - val_acc: 0.9716
Epoch 65/100
60000/60000 [==============================] - 3s 51us/step - loss: 0.1606 - acc: 0.9695 - val_loss: 0.2216 - val_acc: 0.9697
Epoch 66/100
60000/60000 [==============

Epoch 17/100
60000/60000 [==============================] - 3s 48us/step - loss: 0.1379 - acc: 0.9644 - val_loss: 0.1384 - val_acc: 0.9727
Epoch 18/100
60000/60000 [==============================] - 3s 47us/step - loss: 0.1431 - acc: 0.9652 - val_loss: 0.1524 - val_acc: 0.9695
Epoch 19/100
60000/60000 [==============================] - 3s 48us/step - loss: 0.1388 - acc: 0.9660 - val_loss: 0.1382 - val_acc: 0.9729
Epoch 20/100
60000/60000 [==============================] - 3s 48us/step - loss: 0.1368 - acc: 0.9655 - val_loss: 0.1487 - val_acc: 0.9700
Epoch 21/100
60000/60000 [==============================] - 3s 48us/step - loss: 0.1393 - acc: 0.9655 - val_loss: 0.1456 - val_acc: 0.9701
Epoch 22/100
60000/60000 [==============================] - 3s 48us/step - loss: 0.1342 - acc: 0.9676 - val_loss: 0.1785 - val_acc: 0.9686
Epoch 23/100
60000/60000 [==============================] - 3s 49us/step - loss: 0.1398 - acc: 0.9668 - val_loss: 0.1506 - val_acc: 0.9721
Epoch 24/100
60000/60000 [=

Epoch 76/100
60000/60000 [==============================] - 3s 47us/step - loss: 0.1777 - acc: 0.9674 - val_loss: 0.2186 - val_acc: 0.9700
Epoch 77/100
60000/60000 [==============================] - 3s 47us/step - loss: 0.1743 - acc: 0.9682 - val_loss: 0.2313 - val_acc: 0.9694
Epoch 78/100
60000/60000 [==============================] - 3s 47us/step - loss: 0.1753 - acc: 0.9678 - val_loss: 0.2143 - val_acc: 0.9712
Epoch 79/100
60000/60000 [==============================] - 3s 47us/step - loss: 0.1767 - acc: 0.9689 - val_loss: 0.2202 - val_acc: 0.9691
Epoch 80/100
60000/60000 [==============================] - 3s 47us/step - loss: 0.1811 - acc: 0.9663 - val_loss: 0.2190 - val_acc: 0.9679
Epoch 81/100
60000/60000 [==============================] - 3s 47us/step - loss: 0.1729 - acc: 0.9687 - val_loss: 0.2281 - val_acc: 0.9701
Epoch 82/100
60000/60000 [==============================] - 3s 47us/step - loss: 0.1826 - acc: 0.9671 - val_loss: 0.2155 - val_acc: 0.9692
Epoch 83/100
60000/60000 [=

Epoch 34/100
60000/60000 [==============================] - 3s 47us/step - loss: 0.1439 - acc: 0.9679 - val_loss: 0.1552 - val_acc: 0.9719
Epoch 35/100
60000/60000 [==============================] - 3s 47us/step - loss: 0.1415 - acc: 0.9691 - val_loss: 0.1565 - val_acc: 0.9735
Epoch 36/100
60000/60000 [==============================] - 3s 48us/step - loss: 0.1412 - acc: 0.9684 - val_loss: 0.1491 - val_acc: 0.9729
Epoch 37/100
60000/60000 [==============================] - 3s 48us/step - loss: 0.1436 - acc: 0.9680 - val_loss: 0.1594 - val_acc: 0.9719
Epoch 38/100
60000/60000 [==============================] - 3s 49us/step - loss: 0.1507 - acc: 0.9680 - val_loss: 0.1414 - val_acc: 0.9718
Epoch 39/100
60000/60000 [==============================] - 3s 51us/step - loss: 0.1663 - acc: 0.9649 - val_loss: 0.1574 - val_acc: 0.9717
Epoch 40/100
60000/60000 [==============================] - 3s 49us/step - loss: 0.1593 - acc: 0.9665 - val_loss: 0.1571 - val_acc: 0.9723
Epoch 41/100
60000/60000 [=

Epoch 93/100
60000/60000 [==============================] - 3s 49us/step - loss: 0.2107 - acc: 0.9658 - val_loss: 0.2323 - val_acc: 0.9726
Epoch 94/100
60000/60000 [==============================] - 3s 49us/step - loss: 0.2132 - acc: 0.9666 - val_loss: 0.2112 - val_acc: 0.9689
Epoch 95/100
60000/60000 [==============================] - 3s 49us/step - loss: 0.2080 - acc: 0.9659 - val_loss: 0.2426 - val_acc: 0.9699
Epoch 96/100
60000/60000 [==============================] - 3s 47us/step - loss: 0.2042 - acc: 0.9669 - val_loss: 0.2177 - val_acc: 0.9705
Epoch 97/100
60000/60000 [==============================] - 2s 41us/step - loss: 0.2033 - acc: 0.9666 - val_loss: 0.2224 - val_acc: 0.9724
Epoch 98/100
60000/60000 [==============================] - 3s 46us/step - loss: 0.2097 - acc: 0.9661 - val_loss: 0.2299 - val_acc: 0.9716
Epoch 99/100
60000/60000 [==============================] - 3s 47us/step - loss: 0.2158 - acc: 0.9672 - val_loss: 0.2248 - val_acc: 0.9705
Epoch 100/100
60000/60000 [

Epoch 51/100
60000/60000 [==============================] - 3s 54us/step - loss: 0.1839 - acc: 0.9662 - val_loss: 0.2173 - val_acc: 0.9708
Epoch 52/100
60000/60000 [==============================] - 3s 48us/step - loss: 0.1763 - acc: 0.9660 - val_loss: 0.2039 - val_acc: 0.9715
Epoch 53/100
60000/60000 [==============================] - 3s 48us/step - loss: 0.1773 - acc: 0.9659 - val_loss: 0.2099 - val_acc: 0.9711
Epoch 54/100
60000/60000 [==============================] - 3s 47us/step - loss: 0.1734 - acc: 0.9668 - val_loss: 0.2115 - val_acc: 0.9693
Epoch 55/100
60000/60000 [==============================] - 3s 48us/step - loss: 0.1785 - acc: 0.9651 - val_loss: 0.2112 - val_acc: 0.9698
Epoch 56/100
60000/60000 [==============================] - 3s 47us/step - loss: 0.1740 - acc: 0.9656 - val_loss: 0.2067 - val_acc: 0.9705
Epoch 57/100
60000/60000 [==============================] - 3s 47us/step - loss: 0.1789 - acc: 0.9665 - val_loss: 0.2115 - val_acc: 0.9688
Epoch 58/100
60000/60000 [=

60000/60000 [==============================] - 3s 58us/step - loss: 0.1493 - acc: 0.9589 - val_loss: 0.1138 - val_acc: 0.9730
Epoch 9/100
60000/60000 [==============================] - 3s 58us/step - loss: 0.1441 - acc: 0.9617 - val_loss: 0.1248 - val_acc: 0.9718
Epoch 10/100
60000/60000 [==============================] - 3s 54us/step - loss: 0.1440 - acc: 0.9619 - val_loss: 0.1261 - val_acc: 0.9714
Epoch 11/100
60000/60000 [==============================] - 3s 57us/step - loss: 0.1433 - acc: 0.9615 - val_loss: 0.1314 - val_acc: 0.9704
Epoch 12/100
60000/60000 [==============================] - 4s 60us/step - loss: 0.1442 - acc: 0.9624 - val_loss: 0.1198 - val_acc: 0.9713
Epoch 13/100
60000/60000 [==============================] - 4s 62us/step - loss: 0.1379 - acc: 0.9639 - val_loss: 0.1266 - val_acc: 0.9728
Epoch 14/100
60000/60000 [==============================] - 4s 60us/step - loss: 0.1381 - acc: 0.9635 - val_loss: 0.1277 - val_acc: 0.9722
Epoch 15/100
60000/60000 [===============

60000/60000 [==============================] - 3s 53us/step - loss: 0.1842 - acc: 0.9667 - val_loss: 0.2044 - val_acc: 0.9703
Epoch 68/100
60000/60000 [==============================] - 3s 54us/step - loss: 0.1928 - acc: 0.9654 - val_loss: 0.2135 - val_acc: 0.9719
Epoch 69/100
60000/60000 [==============================] - 3s 53us/step - loss: 0.1922 - acc: 0.9638 - val_loss: 0.2018 - val_acc: 0.9727
Epoch 70/100
60000/60000 [==============================] - 3s 54us/step - loss: 0.1884 - acc: 0.9653 - val_loss: 0.2278 - val_acc: 0.9712
Epoch 71/100
60000/60000 [==============================] - 3s 58us/step - loss: 0.1858 - acc: 0.9659 - val_loss: 0.1998 - val_acc: 0.9710
Epoch 72/100
60000/60000 [==============================] - 3s 53us/step - loss: 0.1989 - acc: 0.9658 - val_loss: 0.2527 - val_acc: 0.9691
Epoch 73/100
60000/60000 [==============================] - 3s 52us/step - loss: 0.1894 - acc: 0.9668 - val_loss: 0.2238 - val_acc: 0.9700
Epoch 74/100
60000/60000 [==============